In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.executor.memory", "1g")\
        .config("spark.driver.memory", "4g")\
.appName('VIZ') \
 .master("local[*]")\
    .getOrCreate()
from utils import *
from sedona.spark import *
from pyspark.sql.functions import *
SedonaContext.create(spark)

In [36]:
block_groups_inside_zipcodes = spark.read.parquet("data/POLYGON_CONTAINS_POLYGON/BLOCK_GROUPS_INSIDE_ZIPCODES")\
.where(col("zc_id")=="36695")

In [27]:
block_groups_inside_zipcodes.groupBy("zc_id").count().show()

+-----+-----+
|zc_id|count|
+-----+-----+
|36693|   23|
|36695|  171|
|36575|   56|
+-----+-----+



In [30]:
bg_shape

,bg_shape_wkt
0,"POLYGON ((-88.27577699999999 30.638647, -88.27..."
1,"POLYGON ((-88.295908 30.642174999999998, -88.2..."
2,"POLYGON ((-88.28896499999999 30.666918, -88.28..."


In [42]:
zipcode_shape = block_groups_inside_zipcodes.select("zc_shape_wkt").dropDuplicates().toPandas()
zipcode_h3_shape = block_groups_inside_zipcodes.select("zipcode_h3").dropDuplicates().toPandas()
bg_shape = block_groups_inside_zipcodes.select("bg_shape_wkt","lat","long").dropDuplicates().toPandas()
bg_centroid_h3 = block_groups_inside_zipcodes.select("bg_centroid_h3").toPandas()

In [46]:
m = folium.Map(location=[ 30.638, -88.275]) 
add_polygon_to_map(bg_shape,m,"bg_shape_wkt",color="blue",opacity=0.01, line_weight=3)
add_polygon_to_map(bg_centroid_h3,m,"bg_centroid_h3",color="green",opacity=0.02, line_weight=2)
add_circle_to_map(bg_shape,m,"lat","long",color="red")
m

In [34]:
m = folium.Map(location=[30.626, -88.190]) 
add_polygon_to_map(zipcode_shape,m,"zc_shape_wkt",color="blue",opacity=0.01, line_weight=2.5)
add_polygon_to_map(zipcode_h3_shape,m,"zipcode_h3",color="red",opacity=0.1, line_weight=1.5)
# add_circle_to_map(points_in_block_groups_2,m,"lat","long",color="red")
m

In [47]:
m = folium.Map(location=[30.626, -88.190]) 
add_polygon_to_map(zipcode_shape,m,"zc_shape_wkt",color="blue",opacity=0.01, line_weight=2.5)
add_polygon_to_map(bg_shape,m,"bg_shape_wkt",color="red",opacity=0.1, line_weight=1.5)
# add_circle_to_map(points_in_block_groups_2,m,"lat","long",color="red")
m

In [48]:
m = folium.Map(location=[30.626, -88.190]) 
add_polygon_to_map(bg_centroid_h3,m,"bg_centroid_h3",color="green",opacity=0.02, line_weight=2)
add_polygon_to_map(zipcode_h3_shape,m,"zipcode_h3",color="red",opacity=0.1, line_weight=1.5)
# add_circle_to_map(points_in_block_groups_2,m,"lat","long",color="red")
m